
#### This notebook demonstrates how you can  build a RAG (Retrieval Augmented Generation) with GitHub issues using HuggingFaceH4/zephyr-7b-beta model, and LangChain.
Important Note: You need activate T4-GPU on colab




# 0. Instal dependencies

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00


In [ ]:

!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=5e6e12ba045add00c1f1d0030bbc56453beab9ff3afa1fdce8186a55ea96ab59
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Load Documents


In [ ]:
#In the path file put the documents that you need to RAG
from langchain_community.document_loaders import DirectoryLoader

path = '/content/drive/MyDrive/data'
loader = DirectoryLoader(path, glob="**/*.txt")
docs = loader.load()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
#len(docs)

# 2. Vector base from documents
The most common and straightforward approach to chunking is to define a fixed size of chunks and whether there should be any overlap between them. Keeping some overlap between chunks allows us to preserve some semantic context between the chunks. The recommended splitter for generic text is the RecursiveCharacterTextSplitter, and that's what we'll use here.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)


Now that the docs are all of the appropriate size, we can create a database with their embeddings. This vector database is local storaged to avoid always create this base (bd).

To create document chunk embeddings we'll use the HuggingFaceEmbeddings and the "sentence-transformers/all-MiniLM-L6-v2" embeddings model.
To create the vector database, we'll use FAISS that offers efficient similarity search and clustering of dense vectors, which is what we need here.
We'll access both the embeddings model and FAISS via LangChain API.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# run this, if you do not the vector base or if you changed the documents that you want to load.
db = FAISS.from_documents(chunked_docs, embeddings)
db_path = '/content/drive/MyDrive/data/vetor_db'
db.save_local(db_path)


In [ ]:
db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

We need a way to return(retrieve) the documents given an unstructured query. For that, we'll use the as_retriever method using the db as a context:

search_type="similarity" means we want to perform similarity search between the query and documents
search_kwargs={'k': 3} instructs the retriever to return top 3 results.

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

# 3. Load the llm model
For this example, we chose HuggingFaceH4/zephyr-7b-beta, a small but powerful model.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

First, create a text_generation pipeline using the loaded model and its tokenizer.

Next, create a prompt template
Important: Teh prompt template should follow the format of the model, so if you substitute the model checkpoint, make sure to use the appropriate formatting.

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.01,
    do_sample=True,
    repetition_penalty=1.1,
    #return_full_text=True,
    max_new_tokens=400,
)

#Answer the question based on your knowledge. Use the following context to help:
#Use the following pieces of information to answer the user's question. Pretend you are a customer support agent. When you answer, you must be formal and have soft skills to ensure the best customer experience
#If you don't know the answer, just say that you don't know, don't try to make up an answer


llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Pretend you are a customer support agent. When you answer, you must be formal and have soft skills to ensure the best customer experience.
This is the past context of the conversation:
-"Hello, good day"
-"Thank you for choosing our products and services. We hope that today finds you well. Our team at [Company Name] is committed to providing exceptional customer service and support to all of our clients. Please do not hesitate to reach out to us if you require any assistance or have any queries regarding our offerings."
-"I am having problems with the wifi connection of my smartlock and i dont know what is going on whit it"
-"We apologize for any inconvenience you may be experiencing with the Wi-Fi connection of your August Smart Lock. Our technical support team is available to assist you in resolving this issue. Please provide us with some additional details regarding the nature of the problem so we can better understand the situation and offer appropriate solutions"



Answer the question based on your knowledge. Use the following context to help:
{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-15-5c658a7ae5ce>, line 7)

In [ ]:

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Finally, we need to combine the llm_chain with the retriever to create a RAG chain. We pass the original question through to the final generation step, as well as the retrieved context docs:

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [ ]:
question = "We apologize for any inconvenience you may be experiencing with the Wi-Fi connection of your August Smart Lock. Our technical support team is available to assist you in resolving this issue. Please provide us with some additional details regarding the nature of the problem so we can better understand the situation and offer appropriate solutions"
response = rag_chain.invoke(question)

printing the answer

In [ ]:
response_split = response.split("\n\n")

In [ ]:
stripped_resp = response_split[3]
stripped_resp

In [ ]:
source_docs = response_split[1]
source_docs